In [0]:
# from google.colab import drive
# drive.mount("/content/gdrive/")
# %cd /content/gdrive/My\ Drive/work_space/Chatbot/NLU/Mark1
lang = "en"
path = "../dataset"

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).
/content/gdrive/My Drive/work_space/Chatbot/NLU/Mark1


In [0]:
import pandas as pd
import glob 
import os 
import re
import json
import numpy as np
lang = 0
path = 0

In [0]:
def __docs_loader(intent):
  docs = pd.DataFrame(columns=["text" ,"intent"])
  for uttr in intent["utterances"][lang]:
    text = re.sub("\[[^\]]*\]|\((.*?)\)","", uttr)
    docs = docs.append(pd.Series({"text":text ,"intent":intent["name"]}),ignore_index=True)
  return docs

In [0]:
def __ngram_loader(intent):
  intent_ngram = pd.DataFrame(columns=["text","intent"])
  text = " ".join(intent["utterances"][lang])
  text = re.sub("\[[^\]]*\]","", text)
  intent_ngram = intent_ngram.append(pd.Series({"text":text ,"intent":intent["name"]}),ignore_index=True) if len(intent["slots"])!= 0  else intent_ngram
  return intent_ngram

In [0]:
def __slots_loader(intent):
  slots = pd.DataFrame(columns=["id","name","entities" ,"intent","ngrams"])
  for sl in intent["slots"]:
    serie = {}
    serie["intent"] = intent["name"]
    for col in slots.columns[:-2]:
      serie[col] = sl[col]
    slots = slots.append(pd.Series(serie),ignore_index=True)
  return slots

In [0]:
def __entities_loader(entity):
  entities = pd.DataFrame(columns=["name","type","pattern","examples","occurrences"])
  serie = {}
  for col in entities.columns:
    serie[col] = entity[col]
  entities = entities.append(pd.Series(serie),ignore_index=True)
  return entities


In [0]:
def loader(**kywrd):
  global lang
  global path

  lang = kywrd["lang"]
  path = kywrd["path"]

  for intent_path in glob.glob(os.path.join(path,"intents","*.json")):
    with open(intent_path ,"r") as jf:
      intent = json.load(jf)
    
    try:
      docs=pd.concat((docs , __docs_loader(intent)),axis=0)
      intent_ngram=pd.concat((intent_ngram,__ngram_loader(intent)),axis=0)
      slots=pd.concat((slots, __slots_loader(intent)),axis=0)
    except:
      docs=__docs_loader(intent)
      intent_ngram=__ngram_loader(intent)
      slots= __slots_loader(intent)
    
    
  for entity_path in glob.glob(os.path.join(path,"entities","*.json")):
    with open(entity_path ,"r") as jf:
      entity = json.load(jf)
    try:
      entities=pd.concat((entities , __entities_loader(entity)),axis=0)
    except:
      entities= __entities_loader(entity)
  
  return docs, entities ,slots,intent_ngram

#docs, entities ,slots ,intent_ngram= loader(lang = "en" ,path= "../dataset")